In [ ]:
#Run the setup script
!chmod +x setup.sh
!./setup.sh

In [ ]:
#Run the demo
!python code_generation.py --prompts_file_name "openai_humaneval.jsonl" --model_name "Salesforce/codegen-350M-mono" --generation_strategy "ammarnasr/pass_at_1_gen_config" --output_file_name "codegen350Mmono_pass1_humaneval_python.jsonl" --device "cuda" 

In [8]:
outputs = []
max_num_return_sequences = 10
target_return_sequences = 107

size_of_batch = max_num_return_sequences
num_of_batches = target_return_sequences // size_of_batch
size_of_last_batch = target_return_sequences % size_of_batch


for i in range(num_of_batches):
    output = list(range(size_of_batch))
    outputs.append(output)
    if size_of_last_batch != 0 and i == num_of_batches-1:
        print("last batch")
        output = list(range(size_of_last_batch))
        outputs.append(output)

#concatenate the outputs
outputs = [item for sublist in outputs for item in sublist]


print(len(outputs))

last batch
107
